In [ ]:
import pandas as pd
import re
import json
import numpy as np
import scipy
import nltk

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.style.use('ggplot')

#NASA color palette
nasa = {'red':'#fc3d21','blue':'#0b3d91','grey':'#79797c','black':'#000000'}

# Data

### Import PRS pickle

In [ ]:
df_prs = pd.read_pickle("../../General/Data/prs_index.pkl")

### Import HTML escape characters

In [ ]:
df_html_escape_chars = pd.read_csv('../Data/html_escape_characters.csv')
re_html_escape_chars = "|".join(list(df_html_escape_chars['escape_char']))

# Clean text

In [ ]:
#Freeform text columns that potentially contain "safing"
cols_ff_text = ['title',
                'description',
                'correctiveAction',
                'verificationAnalysis',
                'issues',
#                 'relatedDocuments',
#                 'analysisImpacts',
#                 'attachedFiles',
                'testVerification',
                'executiveSummary',
#                 'procedure',
#                 'rev',
#                 'cogEClosurePlan',
#                 'paragraph',
#                 'rationale',
#                 'cmfFileErrorDescription',
#                 'cmfFileContributingCause',
#                 'cmfFileProximateCause',
#                 'cmfFileCorrectiveAction',
#                 'cmfFileRootCause'
               ]

# cols_ff_text = ['title','description']

### Combine and clean the free-form text fields

In [ ]:
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

In [ ]:
def combine_text_fields(row):
    #Join all text in a report into a single string
    text = ''
    for col in cols_ff_text:
        try:
            text += ' ' + row[col]
        except TypeError:
            continue
    return text

def clean_text(row):
    #Remove HTML escape charactes
    text = re.sub(re_html_escape_chars, ' ', row[col_name])

    #Push text to lowercase
    text = text.lower()

    #Replace any punctuation, special characters, etc. with whitespace     
    text = re.sub('[^A-Za-z0-9]',' ',text)
    
    #Split text based on whitespace
    text = text.split()
    
    #Keep alphanumerics
    text = [re.sub('^(?=[0-9]).*', ' ', word) for word in text]

    #Remove subjectively small text
    text = [word for word in text if len(word) > 2 and not word.isspace()]

    #Remove stopwords
    text = [word for word in text if word not in stopwords.words('english')]
    
    #Set stemmer and use it to stem individual text
#     st = LancasterStemmer()
    st = PorterStemmer()
    text = [st.stem(word) for word in text]
    
    #Return a string of cleaned text
    return ' '.join(text)


#Apply to main subset
# cols_ff_text = ['title','description']
col_name = 'text'
df_prs[col_name] = df_prs.apply(combine_text_fields, axis=1)
df_prs['{0}_cleaned'.format(col_name)] = df_prs.apply(clean_text, axis=1)


df_prs.shape

In [ ]:
#Save cleaned text
# df_prs[['anomalyID','text_cleaned']].to_csv('../../General/Data/text_cleaned_all_ff_text_fields_all_reports.csv',index=False)

In [ ]:
#Load save cleaned text
# df_text_cleaned = pd.read_csv('../../General/Data/text_cleaned_all_ff_text_fields_all_reports.csv')

df_text_cleaned.shape

In [ ]:
print df_prs.shape
print df_text_cleaned.shape

df_prs = pd.merge(left=df_prs,right=df_text_cleaned,how='left',left_on='anomalyID',right_on='anomalyID')
df_prs.shape

In [ ]:
pd.set_option('max_colwidth', 1000)

#Print some output
# df_prs[['title','description','words']]
# df_prs[['fullname',"responsibleEditorFullName","responsibleEditorUserName","assigneeFullName",'description','words']]
df_prs[['text_cleaned']].head()
df_prs.ix[df_prs['anomalyID']==6856,['text','text_cleaned']]

# Term frequencies

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, ENGLISH_STOP_WORDS
from scipy.spatial.distance import cosine

In [ ]:
def get_tfidf_cosine_matrix(report_type, max_n_gram):
    #Subset data
    df_modeling = df_prs[df_prs['reportType']==report_type].copy()
    df_modeling.reset_index(inplace=True)

    #Get term frequencies inverse document frequencies
    tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, max_n_gram))
    tfidf = tfidf_vectorizer.fit_transform(df_modeling['text_cleaned'])

    print "Unique terms:\t{0}".format(len(tfidf_vectorizer.get_feature_names()))
    
    return (tfidf * tfidf.T).A

In [ ]:
#Fix encoding problem
def change_text(row):
    try:
        return row[col].encode('utf-8')
    except AttributeError:
        return ''


def top_n_most_similar(anomaly_id, tf_idf_cosine_dist_matix, n_reports, export_bool=False):
    #Get report type
    report_type = df_prs.ix[df_prs['anomalyID']==anomaly_id,'reportType'].values[0]
    
    #Subset data
    df_modeling = df_prs[df_prs['reportType']==report_type].copy()
    df_modeling.reset_index(inplace=True)
    
    #Get the index associated with the provided anomaly id
    anomaly_id_index = df_modeling[df_modeling['anomalyID']==anomaly_id].index[0]
    
    #Create df from matrix row associated with anomaly id
    df_tf_idf_cosine = pd.DataFrame(tf_idf_cosine_dist_matix[anomaly_id_index],columns=['cosine_dist'])
    
    #Get top-n distance including the anomaly id in question
    top_n = df_tf_idf_cosine.sort_values('cosine_dist',ascending=False).head(n_reports + 1)

    #Merge original PFR data to top-n reports
    df_out = pd.merge(left=top_n, right=df_modeling, how='left', left_index=True, right_index=True)
#     df_out = df_out[['anomalyID','cosine_dist','projectName','title']]
    
    #Create Excel hyperlink 
    base_url = 'prs.jpl.nasa.gov/view_anomaly.asp?smode=pop&iAnomalyID='
    if report_type == "ISA":
        base_url = "https://prs.jpl.nasa.gov/NET/ISAReadOnly.aspx?smode=pop&iAnomalyID="
    elif report_type == "PFR":
        base_url = "https://prs.jpl.nasa.gov/NET/PFRReadOnly.aspx?smode=pop&iAnomalyID="
    df_out['ID'] = df_out.apply(lambda row: '=hyperlink("{0}{1}","{1}")'.format(base_url,row['anomalyID']), axis=1)
    
    #Reset index
    df_out = df_out.reset_index(drop=True)
#     df_out = df_out.reset_index(drop=True)
#     df_out = df_out.rename(columns={"index":"rank"})
    
#     col = 'title'
#     df_out[col] = df_out.apply(change_text, axis=1)
    
    if export_bool:
        #Rename columns
        df_out = df_out.rename(columns={'cosine_dist':'Cosine Distance','projectName':'Project Name','title':'Title'})
        #Create final df and save it
        df_out = df_out[['ID','Cosine Distance','Project Name','Title']]
        df_out.to_csv('../Reports/{0}/anomaly_{1}_top_{2}.csv'.format(report_type, anomaly_id, n_reports), index=True, encoding='utf-8')
    else:
        return df_out[['anomalyID','cosine_dist','projectName','title','date']]

# PFRs

In [ ]:
tfidf_grams_2_prs = get_tfidf_cosine_matrix('PFR',2) #1664262

In [ ]:
tfidf_grams_3_prs = get_tfidf_cosine_matrix('PFR',3) #5607061

In [ ]:
anomaly_id = 59283
top_n_most_similar(anomaly_id, tfidf_grams_2_prs, 15, False)

In [ ]:
top_n_most_similar(anomaly_id, tfidf_grams_3_prs, 15, True)

In [ ]:
#TEST
# df_prs.ix[df_prs['anomalyID']==59283,['text','text_cleaned']]

# ISAs

In [ ]:
'''
Unique terms:	1041724
Unique terms:	3047974
'''
tfidf_grams_2_isa = get_tfidf_cosine_matrix('ISA',2)
tfidf_grams_3_isa = get_tfidf_cosine_matrix('ISA',3)

In [ ]:
top_n_most_similar(59342, tfidf_isa, 10, False)

In [ ]:
top_n_most_similar(58426, tfidf_isa, 10, True)

In [ ]:
#TEST
tfidf_grams_1_isa = get_tfidf_cosine_matrix('ISA',1)

### ISA tests for BW

In [ ]:
'''
58426 is a redo of 58374

58426
    58374 - 1st
    
58374
    58426 - 1st
'''
x = top_n_most_similar(58374, tfidf_grams_2_isa, 5, False)
print x[x['anomalyID']==58426].index[0]
x

top_n_most_similar(58426, tfidf_grams_2_isa, 5, True)
top_n_most_similar(58374, tfidf_grams_2_isa, 5, True)
x

In [ ]:
'''
58425 is a redo of 51700 and 58793

58425
    58793 - 1st
    51700 - 13th
    
51700
    58425 - 21
    58793 - 13

58793
    58425 - 1
    51700 - 11
'''
x = top_n_most_similar(58425, tfidf_grams_2_isa, 200, False)
x[x['anomalyID']==51700].index[0]

top_n_most_similar(58425, tfidf_grams_2_isa, 13, True)
top_n_most_similar(51700, tfidf_grams_2_isa, 21, True)
top_n_most_similar(58793, tfidf_grams_2_isa, 11, True)

In [ ]:
'''
52840 and 44617
Unrelated, but similar words:  52840 and 51010

52840
    44617 - 2nd
    51010 - 15933 th
    
44617
    52840 - 61st
    51010 - 8588th
    
51010
    52840 - 21309
    44617 - 11482
'''
# x = top_n_most_similar(51010, tfidf_grams_2_isa, 5, False)
# x[x['anomalyID']==44617].index[0]

#OUT
top_n_most_similar(52840, tfidf_grams_2_isa, 15934, True)
top_n_most_similar(44617, tfidf_grams_2_isa, 8588, True)

In [ ]:
'''
59337, 57116 and 59246

59337
    57116 - 20th
    59246 - 1st

57116
    59337 - 16th
    59246 - 7th

59246
    57116 - 13th
    59337 - 1st
    
Original performance
#Related: 59337, 57116 and 59246
x = top_n_most_similar(59337, tfidf_grams_2_isa, 400, False).reset_index() #7th and 118th
# x = top_n_most_similar(57116, tfidf_grams_2_isa, 400, False).reset_index() #7th and 13th
x = top_n_most_similar(59246, tfidf_grams_2_isa, 400, False).reset_index() #203rd and 8th
'''
# x = top_n_most_similar(59246, tfidf_grams_2_isa, 20000, False)
# x[x['anomalyID']==59337].index[0]
top_n_most_similar(59337, tfidf_grams_2_isa, 20, True)
top_n_most_similar(57116, tfidf_grams_2_isa, 16, True)
top_n_most_similar(59246, tfidf_grams_2_isa, 13, True)

In [ ]:
'''
57007 and 49280

57007
    49280 - 1st
    
49280
    57007 - 1st
    
Original performance
#Related: 57007 and 49280
x = top_n_most_similar(57007, tfidf_grams_2_isa, 400, False).reset_index() #2nd
x = top_n_most_similar(49280, tfidf_grams_2_isa, 400, False).reset_index() #4th
'''
# x = top_n_most_similar(49280, tfidf_grams_2_isa, 20000, False)
# x[x['anomalyID']==57007].index[0]
top_n_most_similar(57007, tfidf_grams_2_isa, 5, True)
top_n_most_similar(49280, tfidf_grams_2_isa, 5, True)

# Scoring model with related documents

In [ ]:
df_prs.ix[df_prs['anomalyID'].isin([52840,44617,59337,57116,59246,57007,49280,58426,58374,58425,51700,58793]), ['anomalyID','relatedDocuments']]


In [ ]:
'''
58425 is a redo of 51700 and 58793

58425
    58793 - 1st
    51700 - 13th
    
51700
    58425 - 21
    58793 - 13

58793
    58425 - 1
    51700 - 11
'''
x = top_n_most_similar(58793, tfidf_grams_2_isa, 200, False)
x[x['anomalyID']==51700].index[0]

In [ ]:
print df_prs.ix[(df_prs['reportType']=='ISA')].shape

print df_prs.ix[(df_prs['reportType']=='ISA')&
          (df_prs['relatedDocuments'].str.contains("(?i)ISA")),['anomalyID','projectName','relatedDocuments']].shape

print df_prs.ix[(df_prs['reportType']=='ISA')&
          (df_prs['relatedDocuments'].str.contains("(?i)PFR")),['anomalyID','projectName','relatedDocuments']].shape

In [ ]:
df_prs.shape

In [ ]:
# print df_prs[(df_prs['reportType']=='ISA')].shape
print df_prs[(df_prs['reportType']=='ISA')&(~df_prs['relatedDocuments'].isnull())].shape 
print df_prs[(df_prs['reportType']=='ISA')&(df_prs['relatedDocuments'].str.contains("(?i)related"))].shape 

df_prs.ix[(df_prs['projectName']=='Mars Science Lab')&(df_prs['reportType']=='ISA')&(~df_prs['relatedDocuments'].isnull()),['anomalyID','projectName','relatedDocuments']]
df_prs.ix[(df_prs['projectName']=='Mars Science Lab')&
          (df_prs['reportType']=='ISA')&
          (df_prs['relatedDocuments'].str.contains("(?i)related")),['anomalyID','projectName','relatedDocuments']]

# TEST

In [ ]:
df_prs.ix[df_prs['anomalyID']==10091,[col for col in df_prs.columns if re.search('(?i)name',col)]]

In [ ]:
df_prs.ix[df_prs['title'].str.contains("::"),'date'].describe()

In [ ]:
import pandas as pd
from os import listdir
from os.path import isfile, join

# onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
base_path = '../Reports/Document_Similarity_Ranks/'
df_scores = pd.DataFrame()
for score_file in listdir(base_path):
    df_temp = pd.read_csv(base_path+score_file)
    df_scores = pd.concat([df_scores, df_temp])

In [ ]:
df_scores.describe()#.plot(kind='hist',bins=100,figsize=(15,7))

In [ ]:
df_scores.describe()#.plot(kind='hist',bins=100,figsize=(15,7))

In [ ]:
print df_scores[df_scores['rank'].isnull()].shape
print df_scores[df_scores['rank']<=20].shape
print df_scores[df_scores['rank']>=1000].shape
df_scores[df_scores['rank']<=10].shape[0] / float(df_scores.shape[0])

In [ ]:
df_scores[['rank']].plot(kind='hist',figsize=(15,8),bins=100)
df_scores.ix[df_scores['rank']<=50,['rank']].plot(kind='hist',figsize=(15,8),bins=100)
print df_scores.ix[df_scores['rank']<=10,['rank']].shape
print df_scores.ix[df_scores['rank']>10,['rank']].shape

In [ ]:
df_scores.shape#.groupby('rank').count()

In [ ]:
df_scores.sort_values('rank',ascending=False)
df_scores[df_scores['rank'].isnull()].shape

In [ ]:
df_scores[df_scores['anomaly_id']==10091]

In [ ]:
df_scores['rank'].isnull().shape